## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import os
from geopy.geocoders import GoogleV3
import geocoder
from shapely.geometry import Point
import glob

## Data read-in

In [2]:
file_list = [file for file in glob.glob('*.csv')]

In [3]:
file_list

['Top West township assessments - office.csv',
 'Top West township assessments - industrial.csv',
 'Top West township assessments - multifamily.csv']

In [4]:
df_office = pd.read_csv(file_list[0])
df_industrial = pd.read_csv(file_list[1])
df_multifamily = pd.read_csv(file_list[2])

In [5]:
df_office = df_office.drop(columns='name')

In [6]:
df_office = df_office.rename(columns={'2021 vaue':'2021 value','2024 value ':'2024 value',
                                      'percent change in initial assessment ':'percent change in initial assessment'})

In [7]:
df_industrial = df_industrial.rename(columns={'2021 vaue':'2021 value','2024 value ':'2024 value',
                                      'percent change in initial assessment ':'percent change in initial assessment'})

In [8]:
df_multifamily = df_multifamily.rename(columns={'percent change in initial assessment ':'percent change in initial assessment'})

In [9]:
df_industrial.columns

Index(['PIN', 'category', 'address', '2021 value', '2024 value',
       'percent change in initial assessment'],
      dtype='object')

In [10]:
df = pd.concat([df_office,df_industrial,df_multifamily])

In [11]:
df

,PIN,category,address,2021 value,2024 value,percent change in initial assessment
0,17-16-130-001-0000,office,433 West Van Buren,"$854,168,112","$897,138,000",5.0%
1,17-16-120-010-0000,office,310 South Canal,"$20,069,100","$532,711,000",NaN
2,17-09-326-004-0000,office,400 West Lake,"$505,992,478","$449,413,000",-11.2%
3,17-09-342-002-0000,office,500 West Madison,"$546,764,886","$412,446,000",-24.6%
4,17-09-306-033-0000,office,444 West Lake,"$432,612,009","$396,070,000",-8.4%
0,17-05-306-007-0000,industrial,1220 West Cortez,"$15,110,480","$153,010,000",913%
1,16-35-300-039-0000,industrial,3501 South Pulaski,"$17,945,328","$113,828,000",534%
2,17-09-100-004-0000,industrial,655 West Chicago,"$76,519,928","$43,400,000",-43%
3,16-24-223-001-0000,industrial,2558 West 16th,"$64,839,414","$56,178,000",-13%
4,16-16-310-011-0000,industrial,5490 West Roosevelt,"$45,751,016","$42,784,000",-6%


## Data work

In [12]:
df.columns

Index(['PIN', 'category', 'address', '2021 value', '2024 value',
       'percent change in initial assessment'],
      dtype='object')

In [13]:
df[['2021 value int', '2024 value int']] = df[['2021 value', '2024 value']].replace({'\$': '', ',': ''}, 
                                                                                    regex=True)

df[['2021 value int', '2024 value int']] = df[['2021 value int', '2024 value int']].astype(int)

In [14]:
df['percent change in initial assessment int'] = df['percent change in initial assessment'].replace('%',regex=True)

In [15]:
df['address_full'] = df['address'] + ' Chicago, IL'

In [16]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [17]:
df['geocoded'] = df['address_full'].apply(geolocator.geocode, timeout=10)

In [18]:
df['point'] = df['geocoded'].apply(lambda loc: tuple(loc.point) if loc else None)

In [19]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [20]:
df['category'] = df['category'].str.title()

In [21]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Optionally, set the coordinate reference system (CRS) for the GeoDataFrame
gdf.set_crs(epsg=4326, inplace=True)  # EPSG 4326 is the standard WGS 84 latitude and longitude system

,PIN,category,address,2021 value,2024 value,percent change in initial assessment,2021 value int,2024 value int,percent change in initial assessment int,address_full,geocoded,point,lat,lon,altitude,geometry
0,17-16-130-001-0000,Office,433 West Van Buren,"$854,168,112","$897,138,000",5.0%,854168112,897138000,5.0%,"433 West Van Buren Chicago, IL","(433 W Van Buren St, Chicago, IL 60607, USA, (...","(41.8765027, -87.6386911, 0.0)",41.876503,-87.638691,0.0,POINT (-87.63869 41.87650)
1,17-16-120-010-0000,Office,310 South Canal,"$20,069,100","$532,711,000",NaN,20069100,532711000,NaN,"310 South Canal Chicago, IL","(310 S Canal St, Chicago, IL 60606, USA, (41.8...","(41.8773101, -87.6400192, 0.0)",41.877310,-87.640019,0.0,POINT (-87.64002 41.87731)
2,17-09-326-004-0000,Office,400 West Lake,"$505,992,478","$449,413,000",-11.2%,505992478,449413000,-11.2%,"400 West Lake Chicago, IL","(400 W Lake St, Chicago, IL 60606, USA, (41.88...","(41.8851033, -87.63864869999999, 0.0)",41.885103,-87.638649,0.0,POINT (-87.63865 41.88510)
3,17-09-342-002-0000,Office,500 West Madison,"$546,764,886","$412,446,000",-24.6%,546764886,412446000,-24.6%,"500 West Madison Chicago, IL","(Accenture Tower, 500 W Madison St, Chicago, I...","(41.8823361, -87.6403662, 0.0)",41.882336,-87.640366,0.0,POINT (-87.64037 41.88234)
4,17-09-306-033-0000,Office,444 West Lake,"$432,612,009","$396,070,000",-8.4%,432612009,396070000,-8.4%,"444 West Lake Chicago, IL","(444 W Lake St, Chicago, IL 60606, USA, (41.88...","(41.8860918, -87.6397677, 0.0)",41.886092,-87.639768,0.0,POINT (-87.63977 41.88609)
0,17-05-306-007-0000,Industrial,1220 West Cortez,"$15,110,480","$153,010,000",913%,15110480,153010000,913%,"1220 West Cortez Chicago, IL","(1220 W Cortez St, Chicago, IL 60642, USA, (41...","(41.9012462, -87.6579593, 0.0)",41.901246,-87.657959,0.0,POINT (-87.65796 41.90125)
1,16-35-300-039-0000,Industrial,3501 South Pulaski,"$17,945,328","$113,828,000",534%,17945328,113828000,534%,"3501 South Pulaski Chicago, IL","(3501 S Pulaski Rd, Chicago, IL 60623, USA, (4...","(41.8308907, -87.72343219999999, 0.0)",41.830891,-87.723432,0.0,POINT (-87.72343 41.83089)
2,17-09-100-004-0000,Industrial,655 West Chicago,"$76,519,928","$43,400,000",-43%,76519928,43400000,-43%,"655 West Chicago Chicago, IL","(655 W Chicago Ave, Chicago, IL 60654, USA, (4...","(41.8959156, -87.6451605, 0.0)",41.895916,-87.645161,0.0,POINT (-87.64516 41.89592)
3,16-24-223-001-0000,Industrial,2558 West 16th,"$64,839,414","$56,178,000",-13%,64839414,56178000,-13%,"2558 West 16th Chicago, IL","(2558 W 16th St, Chicago, IL 60608, USA, (41.8...","(41.8602243, -87.6898621, 0.0)",41.860224,-87.689862,0.0,POINT (-87.68986 41.86022)
4,16-16-310-011-0000,Industrial,5490 West Roosevelt,"$45,751,016","$42,784,000",-6%,45751016,42784000,-6%,"5490 West Roosevelt Chicago, IL","(5490 W Roosevelt Rd, Chicago, IL 60644, USA, ...","(41.8672149, -87.7617156, 0.0)",41.867215,-87.761716,0.0,POINT (-87.76172 41.86721)


In [22]:
gdf[['geocoded','point']] = gdf[['geocoded','point']].astype(str)

In [23]:
# Export to GeoJSON
gdf.to_file("top_west_assessments_06_06_24.geojson", driver='GeoJSON')